# Diagnostic 16: Comprehensive Grid Search - ALL 9 Strategies

**Purpose:** Optimize parameters for ALL trading strategies with small farmer costs

**Strategies:**
1. ImmediateSaleStrategy
2. EqualBatchStrategy
3. PriceThresholdStrategy
4. MovingAverageStrategy
5. PriceThresholdPredictive
6. MovingAveragePredictive
7. ExpectedValueStrategy
8. ConsensusStrategy
9. RiskAdjustedStrategy

**Key Configuration:**
- Small farmer costs: 0.005% storage/day, 0.01% transaction
- Coarse grid for initial optimization
- All strategies imported from all_strategies_pct.py

**Expected Results:**
- Optimal parameters for each strategy
- Clear ranking with realistic costs
- Matched pairs showing prediction value-add

In [ ]:
%run ../00_setup_and_config

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from itertools import product
import importlib.util

print("="*80)
print("DIAGNOSTIC 16: COMPREHENSIVE GRID SEARCH - ALL 9 STRATEGIES")
print("="*80)
print("\nOptimizing ALL strategies with small farmer costs")

## Load Strategies

In [ ]:
# Force fresh reload
if 'all_strategies_pct' in sys.modules:
    del sys.modules['all_strategies_pct']

spec = importlib.util.spec_from_file_location("all_strategies_pct", "all_strategies_pct.py")
strategies_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(strategies_module)

ImmediateSaleStrategy = strategies_module.ImmediateSaleStrategy
EqualBatchStrategy = strategies_module.EqualBatchStrategy
PriceThresholdStrategy = strategies_module.PriceThresholdStrategy
MovingAverageStrategy = strategies_module.MovingAverageStrategy
PriceThresholdPredictive = strategies_module.PriceThresholdPredictive
MovingAveragePredictive = strategies_module.MovingAveragePredictive
ExpectedValueStrategy = strategies_module.ExpectedValueStrategy
ConsensusStrategy = strategies_module.ConsensusStrategy
RiskAdjustedStrategy = strategies_module.RiskAdjustedStrategy

print("✓ Loaded all 9 strategies from all_strategies_pct.py")

## Load Data with Small Farmer Costs

In [ ]:
COMMODITY = 'coffee'
MODEL_VERSION = 'synthetic_acc90'

DATA_PATHS = get_data_paths(COMMODITY, MODEL_VERSION)
COMMODITY_CONFIG = COMMODITY_CONFIGS[COMMODITY]

# OVERRIDE: Small farmer realistic costs
COMMODITY_CONFIG['storage_cost_pct_per_day'] = 0.005  # Was 0.025%
COMMODITY_CONFIG['transaction_cost_pct'] = 0.01       # Was 0.25%

print("Loading data...")
prices_table = get_data_paths(COMMODITY)['prices_prepared']
prices = spark.table(prices_table).toPandas()
prices['date'] = pd.to_datetime(prices['date'])

matrices_path = DATA_PATHS['prediction_matrices']
with open(matrices_path, 'rb') as f:
    prediction_matrices = pickle.load(f)
prediction_matrices = {pd.to_datetime(k): v for k, v in prediction_matrices.items()}

print(f"✓ Loaded {len(prices)} price records")
print(f"✓ Loaded {len(prediction_matrices)} prediction matrices")
print(f"\nSmall Farmer Costs:")
print(f"  Storage: {COMMODITY_CONFIG['storage_cost_pct_per_day']}% per day")
print(f"  Transaction: {COMMODITY_CONFIG['transaction_cost_pct']}%")

## Backtest Engine

In [ ]:
class DiagnosticBacktestEngine:
    """Backtest engine for grid search"""
    
    def __init__(self, prices_df, prediction_matrices, commodity_config):
        self.prices = prices_df
        self.prediction_matrices = prediction_matrices
        self.config = commodity_config
        
    def run_backtest(self, strategy, initial_inventory=50.0):
        inventory = initial_inventory
        trades = []
        total_revenue = 0
        total_transaction_costs = 0
        total_storage_costs = 0
        
        strategy.reset()
        strategy.set_harvest_start(0)
        
        for day in range(len(self.prices)):
            current_date = self.prices.iloc[day]['date']
            current_price = self.prices.iloc[day]['price']
            price_history = self.prices.iloc[:day+1].copy()
            predictions = self.prediction_matrices.get(current_date, None)
            
            decision = strategy.decide(
                day=day,
                inventory=inventory,
                current_price=current_price,
                price_history=price_history,
                predictions=predictions
            )
            
            if decision['action'] == 'SELL' and decision['amount'] > 0:
                amount = min(decision['amount'], inventory)
                price_per_ton = current_price * 20
                revenue = amount * price_per_ton
                transaction_cost = revenue * (self.config['transaction_cost_pct'] / 100)
                
                total_revenue += revenue
                total_transaction_costs += transaction_cost
                inventory -= amount
                
                trades.append({
                    'day': day,
                    'amount': amount,
                    'price': current_price,
                    'revenue': revenue
                })
            
            if inventory > 0:
                avg_price = self.prices.iloc[:day+1]['price'].mean()
                price_per_ton = avg_price * 20
                storage_cost = inventory * price_per_ton * (self.config['storage_cost_pct_per_day'] / 100)
                total_storage_costs += storage_cost
        
        net_earnings = total_revenue - total_transaction_costs - total_storage_costs
        
        return {
            'net_earnings': net_earnings,
            'total_revenue': total_revenue,
            'num_trades': len(trades),
            'storage_costs': total_storage_costs,
            'final_inventory': inventory
        }

engine = DiagnosticBacktestEngine(prices, prediction_matrices, COMMODITY_CONFIG)
print("✓ Backtest engine ready")

## Parameter Grids - ALL 9 Strategies

In [ ]:
# Coarse grids for all strategies - COMPLETE PARAMETERIZATION
PARAM_GRIDS = {
    'immediate_sale': {
        'min_batch_size': [3.0, 5.0, 7.0, 10.0],
        'sale_frequency_days': [5, 7, 10, 14]
    },
    
    'equal_batch': {
        'batch_size': [0.15, 0.20, 0.25, 0.30],
        'frequency_days': [20, 25, 30, 35]
    },
    
    'price_threshold': {
        'threshold_pct': [0.02, 0.03, 0.05, 0.07],
        'batch_baseline': [0.20, 0.25, 0.30, 0.35],
        'max_days_without_sale': [45, 60, 75]
    },
    
    'moving_average': {
        'ma_period': [20, 25, 30, 35],
        'batch_baseline': [0.20, 0.25, 0.30],
        'batch_strong_momentum': [0.15, 0.20, 0.25],
        'batch_overbought': [0.25, 0.30, 0.35],
        'batch_overbought_strong': [0.30, 0.35, 0.40],
        'max_days_without_sale': [45, 60, 75]
    },
    
    'price_threshold_predictive': {
        'threshold_pct': [0.02, 0.03, 0.05, 0.07],
        'batch_baseline': [0.20, 0.25, 0.30, 0.35],
        'batch_overbought_strong': [0.30, 0.35, 0.40],
        'batch_overbought': [0.25, 0.30, 0.35],
        'batch_strong_trend': [0.15, 0.20, 0.25],
        'max_days_without_sale': [45, 60, 75],
        'min_net_benefit_pct': [0.3, 0.5, 0.7, 1.0],
        'high_confidence_cv': [0.03, 0.05, 0.08],
        'scenario_shift_aggressive': [1, 2],
        'scenario_shift_conservative': [1, 2]
    },
    
    'moving_average_predictive': {
        'ma_period': [20, 25, 30, 35],
        'batch_baseline': [0.20, 0.25, 0.30],
        'batch_strong_momentum': [0.15, 0.20, 0.25],
        'batch_overbought': [0.25, 0.30, 0.35],
        'batch_overbought_strong': [0.30, 0.35, 0.40],
        'max_days_without_sale': [45, 60, 75],
        'min_net_benefit_pct': [0.3, 0.5, 0.7, 1.0],
        'high_confidence_cv': [0.03, 0.05, 0.08],
        'scenario_shift_aggressive': [1, 2],
        'scenario_shift_conservative': [1, 2]
    },
    
    'expected_value': {
        # Prediction thresholds
        'min_net_benefit_pct': [0.3, 0.5, 0.7, 1.0],
        'negative_threshold_pct': [-0.5, -0.3, -0.1],
        # Confidence thresholds
        'high_confidence_cv': [0.03, 0.05, 0.08],
        'medium_confidence_cv': [0.10, 0.15],
        # Technical indicators
        'strong_trend_adx': [20, 25],
        # Batch sizing (all 5 scenarios)
        'batch_positive_confident': [0.0, 0.05],
        'batch_positive_uncertain': [0.10, 0.15, 0.20],
        'batch_marginal': [0.15, 0.20],
        'batch_negative_mild': [0.25, 0.30],
        'batch_negative_strong': [0.35, 0.40],
        # Timing
        'cooldown_days': [5, 7],
        'baseline_batch': [0.15, 0.20],
        'baseline_frequency': [25, 30]
    },
    
    'consensus': {
        # Consensus thresholds
        'consensus_threshold': [0.60, 0.65, 0.70, 0.75],
        'very_strong_consensus': [0.80, 0.85],
        'moderate_consensus': [0.55, 0.60],
        # Return and benefit thresholds
        'min_return': [0.02, 0.03, 0.04, 0.05],
        'min_net_benefit_pct': [0.3, 0.5, 0.7],
        # Confidence
        'high_confidence_cv': [0.03, 0.05, 0.08],
        # Batch sizing (all 4 scenarios)
        'batch_strong_consensus': [0.0, 0.05],
        'batch_moderate': [0.10, 0.15, 0.20],
        'batch_weak': [0.25, 0.30],
        'batch_bearish': [0.35, 0.40],
        # Timing
        'evaluation_day': [10, 14],
        'cooldown_days': [5, 7]
    },
    
    'risk_adjusted': {
        # Return and benefit thresholds
        'min_return': [0.02, 0.03, 0.04, 0.05],
        'min_net_benefit_pct': [0.3, 0.5, 0.7],
        # Uncertainty (risk) thresholds
        'max_uncertainty_low': [0.03, 0.05, 0.08],
        'max_uncertainty_medium': [0.10, 0.15, 0.20],
        'max_uncertainty_high': [0.25, 0.30, 0.35],
        # Technical indicators
        'strong_trend_adx': [20, 25],
        # Batch sizing (all 4 risk tiers)
        'batch_low_risk': [0.0, 0.05],
        'batch_medium_risk': [0.10, 0.15],
        'batch_high_risk': [0.25, 0.30],
        'batch_very_high_risk': [0.35, 0.40],
        # Timing
        'evaluation_day': [10, 14],
        'cooldown_days': [5, 7]
    }
}

# Calculate total combinations
print("Parameter combinations per strategy:")
total_combos = 0
for strategy, grid in PARAM_GRIDS.items():
    combos = np.prod([len(v) for v in grid.values()])
    print(f"  {strategy}: {combos:,}")
    total_combos += combos

print(f"\nTotal combinations: {total_combos:,}")
print(f"\n⚠️  WARNING: This is a MASSIVE grid search!")
print(f"Consider reducing grid size or running in parallel on Databricks")

## Grid Search Function

In [ ]:
def grid_search_strategy(strategy_class, param_grid, strategy_name, engine, 
                        needs_costs=False):
    """
    Grid search for a single strategy.
    
    Args:
        strategy_class: Strategy class to test
        param_grid: Dictionary of parameter names to lists of values
        strategy_name: Display name
        engine: Backtest engine
        needs_costs: Whether strategy needs cost parameters in __init__
    
    Returns:
        best_params, best_result, all_results
    """
    
    print(f"\n{'='*80}")
    print(f"Grid searching: {strategy_name}")
    print("="*80)
    
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    print(f"Testing {len(combinations):,} combinations")
    
    best_earnings = -float('inf')
    best_params = None
    best_result = None
    all_results = []
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        # Add cost parameters if needed
        if needs_costs:
            params['storage_cost_pct_per_day'] = COMMODITY_CONFIG['storage_cost_pct_per_day']
            params['transaction_cost_pct'] = COMMODITY_CONFIG['transaction_cost_pct']
        
        # Create strategy
        try:
            strategy = strategy_class(**params)
        except Exception as e:
            print(f"  Error creating strategy with {params}: {e}")
            continue
        
        # Run backtest
        try:
            result = engine.run_backtest(strategy)
        except Exception as e:
            print(f"  Error running backtest: {e}")
            continue
        
        # Store result
        all_results.append({
            'params': params.copy(),
            'net_earnings': result['net_earnings'],
            'num_trades': result['num_trades'],
            'storage_costs': result['storage_costs']
        })
        
        # Update best
        if result['net_earnings'] > best_earnings:
            best_earnings = result['net_earnings']
            best_params = params.copy()
            best_result = result
        
        # Progress
        if (i + 1) % 50 == 0 or (i + 1) == len(combinations):
            print(f"  Progress: {i+1}/{len(combinations)} | Best: ${best_earnings:,.0f}")
    
    print(f"\n✓ Best: ${best_earnings:,.2f}")
    print(f"  Trades: {best_result['num_trades']}")
    print(f"  Storage: ${best_result['storage_costs']:,.2f}")
    
    return best_params, best_result, all_results

print("✓ Grid search function ready")

## Run Grid Searches - ALL 9 Strategies

In [ ]:
print("="*80)
print("STARTING COMPREHENSIVE GRID SEARCH")
print("="*80)

all_strategy_results = {}

### 1. Immediate Sale

In [ ]:
params, result, all_res = grid_search_strategy(
    ImmediateSaleStrategy,
    PARAM_GRIDS['immediate_sale'],
    "Immediate Sale",
    engine,
    needs_costs=False
)
all_strategy_results['immediate_sale'] = (params, result, all_res)

### 2. Equal Batch

In [ ]:
params, result, all_res = grid_search_strategy(
    EqualBatchStrategy,
    PARAM_GRIDS['equal_batch'],
    "Equal Batch",
    engine,
    needs_costs=False
)
all_strategy_results['equal_batch'] = (params, result, all_res)

### 3. Price Threshold (Baseline)

In [ ]:
params, result, all_res = grid_search_strategy(
    PriceThresholdStrategy,
    PARAM_GRIDS['price_threshold'],
    "Price Threshold",
    engine,
    needs_costs=False
)
all_strategy_results['price_threshold'] = (params, result, all_res)

### 4. Moving Average (Baseline)

In [ ]:
params, result, all_res = grid_search_strategy(
    MovingAverageStrategy,
    PARAM_GRIDS['moving_average'],
    "Moving Average",
    engine,
    needs_costs=False
)
all_strategy_results['moving_average'] = (params, result, all_res)

### 5. Price Threshold Predictive (Matched Pair)

In [ ]:
params, result, all_res = grid_search_strategy(
    PriceThresholdPredictive,
    PARAM_GRIDS['price_threshold_predictive'],
    "Price Threshold Predictive",
    engine,
    needs_costs=True
)
all_strategy_results['price_threshold_predictive'] = (params, result, all_res)

### 6. Moving Average Predictive (Matched Pair)

In [ ]:
params, result, all_res = grid_search_strategy(
    MovingAveragePredictive,
    PARAM_GRIDS['moving_average_predictive'],
    "Moving Average Predictive",
    engine,
    needs_costs=True
)
all_strategy_results['moving_average_predictive'] = (params, result, all_res)

### 7. Expected Value

In [ ]:
params, result, all_res = grid_search_strategy(
    ExpectedValueStrategy,
    PARAM_GRIDS['expected_value'],
    "Expected Value",
    engine,
    needs_costs=True
)
all_strategy_results['expected_value'] = (params, result, all_res)

### 8. Consensus

In [ ]:
params, result, all_res = grid_search_strategy(
    ConsensusStrategy,
    PARAM_GRIDS['consensus'],
    "Consensus",
    engine,
    needs_costs=True
)
all_strategy_results['consensus'] = (params, result, all_res)

### 9. Risk-Adjusted

In [ ]:
params, result, all_res = grid_search_strategy(
    RiskAdjustedStrategy,
    PARAM_GRIDS['risk_adjusted'],
    "Risk-Adjusted",
    engine,
    needs_costs=True
)
all_strategy_results['risk_adjusted'] = (params, result, all_res)

## Summary Analysis

In [ ]:
print("\n" + "="*80)
print("COMPREHENSIVE RESULTS - ALL 9 STRATEGIES")
print("="*80)

# Create summary
summary = []
for name, (params, result, all_res) in all_strategy_results.items():
    summary.append({
        'strategy': name,
        'net_earnings': result['net_earnings'],
        'num_trades': result['num_trades'],
        'storage_costs': result['storage_costs'],
        'params': params
    })

# Sort by earnings
summary.sort(key=lambda x: x['net_earnings'], reverse=True)

# Display ranking
print("\nRanking:")
print(f"{'Rank':<6} {'Strategy':<30} {'Net Earnings':>15} {'Trades':>8} {'Storage':>12}")
print("-" * 80)

for i, s in enumerate(summary, 1):
    print(f"{i:<6} {s['strategy']:<30} ${s['net_earnings']:>14,.2f} {s['num_trades']:>8} ${s['storage_costs']:>11,.2f}")

# Best baseline
baselines = ['immediate_sale', 'equal_batch', 'price_threshold', 'moving_average']
best_baseline = max([s for s in summary if s['strategy'] in baselines], 
                   key=lambda x: x['net_earnings'])

print(f"\nBest Baseline: {best_baseline['strategy']} (${best_baseline['net_earnings']:,.2f})")

# Prediction strategies vs baseline
print("\n" + "="*80)
print("PREDICTION STRATEGIES vs BEST BASELINE")
print("="*80)

predictions = [s for s in summary if s['strategy'] not in baselines]
for s in predictions:
    improvement = s['net_earnings'] - best_baseline['net_earnings']
    pct = 100 * improvement / best_baseline['net_earnings']
    marker = "✓" if improvement > 0 else "✗"
    print(f"{marker} {s['strategy']:<30} ${s['net_earnings']:>12,.2f} ({pct:+6.2f}%)")

# Matched pairs
print("\n" + "="*80)
print("MATCHED PAIR ANALYSIS")
print("="*80)

pairs = [
    ('price_threshold', 'price_threshold_predictive'),
    ('moving_average', 'moving_average_predictive')
]

for base_name, pred_name in pairs:
    base = all_strategy_results[base_name][1]
    pred = all_strategy_results[pred_name][1]
    improvement = pred['net_earnings'] - base['net_earnings']
    pct = 100 * improvement / base['net_earnings']
    
    print(f"\n{base_name.upper()}:")
    print(f"  Baseline:   ${base['net_earnings']:,.2f}")
    print(f"  Predictive: ${pred['net_earnings']:,.2f}")
    print(f"  Improvement: ${improvement:,.2f} ({pct:+.2f}%)")
    if improvement > 0:
        print(f"  ✓ Predictions add value!")
    else:
        print(f"  ✗ Predictions hurt performance")

## Save Results

In [ ]:
import json

# Prepare summary for JSON
json_summary = {
    'timestamp': datetime.now().isoformat(),
    'commodity': COMMODITY,
    'model_version': MODEL_VERSION,
    'costs': {
        'storage_pct_per_day': COMMODITY_CONFIG['storage_cost_pct_per_day'],
        'transaction_pct': COMMODITY_CONFIG['transaction_cost_pct']
    },
    'strategies': {}
}

for name, (params, result, all_res) in all_strategy_results.items():
    # Remove cost params from display
    display_params = {k: v for k, v in params.items() 
                     if k not in ['storage_cost_pct_per_day', 'transaction_cost_pct']}
    
    json_summary['strategies'][name] = {
        'net_earnings': float(result['net_earnings']),
        'num_trades': result['num_trades'],
        'storage_costs': float(result['storage_costs']),
        'best_params': display_params
    }

# Save JSON
json_path = '/Volumes/commodity/trading_agent/files/diagnostic_16_grid_search_summary.json'
with open(json_path, 'w') as f:
    json.dump(json_summary, f, indent=2)

print(f"✓ Summary saved: {json_path}")

# Save full results as pickle
pkl_path = '/Volumes/commodity/trading_agent/files/diagnostic_16_grid_search_results.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump(all_strategy_results, f)

print(f"✓ Full results saved: {pkl_path}")

print("\n" + "="*80)
print("DIAGNOSTIC 16 COMPLETE")
print("="*80)
print(f"\nOptimized {len(all_strategy_results)} strategies")
print(f"Total combinations tested: {sum(len(r[2]) for r in all_strategy_results.values()):,}")
print(f"\nResults saved to Volume for analysis")